In [1]:
!pip install chromadb langchain openai tiktoken

In [2]:
pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [3]:
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip

In [4]:
!unzip -q new_articles.zip -d new_articles